In [1]:
# from google.colab import drive
# drive.mount("/content/mydrive/")

In [2]:
# cd /content/mydrive/MyDrive/hkinetics_managermdb51/hmdb51

In [1]:
import os

In [2]:
path = "/home/uas-dtu/Desktop/Kinetics"
all_videos = []
all_labels = []
for i in os.listdir(path):
    for j in os.listdir(path + '/' + i):
        all_videos.append(path+'/' + i + '/' + j)
        all_labels.append(i)

In [3]:
all_labels

['catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching or throwing frisbee',
 'catching

In [4]:
import tensorflow as tf
import numpy as np
import random
import cv2
import os,sys
from PIL import Image
import scipy.misc


def ToImg(raw_flow,bound):
    flow = np.array(raw_flow, dtype=np.float32)
    flow[flow>bound] = bound
    flow[flow<-bound] = -bound
    flow -= -bound
    flow *= (1/float(2*bound))
    return flow

def save_flows(flows, bound):
    flow_x = np.expand_dims(ToImg(flows[...,0], bound), axis=-1)
    flow_y = np.expand_dims(ToImg(flows[...,1], bound), axis=-1)
    flow_0 = np.zeros_like(flow_x)
    return np.concatenate((flow_x, flow_y, flow_0), axis=-1)


def capture_trim_resize(path, shape, n_frames):
  out = []
  cap = cv2.VideoCapture(path)
  valid = True
  for i in range(n_frames):
    _, frame = cap.read()
    if not _:
      frame = np.zeros((shape[0], shape[1], 3), dtype=np.uint8)
    frame = cv2.resize(frame, (shape[1], shape[0]))
    out.append(frame)
  return np.array(out)


def tvl(frames, bound):
  output = []
  len_frame=len(frames)
  frame_num = 0
  image, prev_image, gray, prev_gray = None, None, None, None
  num0 = 0
  while True:
    if num0 >= len_frame:
      break
    frame = frames[num0]
    frame = cv2.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2))
    num0 += 1
    if frame_num == 0:
      image = np.zeros_like(frame)
      gray = np.zeros_like(frame)
      prev_gray = np.zeros_like(frame)
      prev_image = frame
      prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)
      frame_num += 1

    image = frame
    gray=cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    frame_0 = prev_gray
    frame_1 = gray
    dtvl1 = cv2.optflow.DualTVL1OpticalFlow_create()
    flowDTVL1 = dtvl1.calc(frame_0, frame_1, None)
    flow = save_flows(flowDTVL1, bound)
    output.append(flow)
    prev_gray = gray
    prev_image = image
    frame_num += 1
  output = np.array(output)
  return tf.image.resize(output, [frames.shape[1], frames.shape[2]]).numpy()


class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, x_data, y_data, frame_shape, n_frames, batch_size, shuffle=True, flow_bound=15):
#     self.path = videos_folder_path
    self.n_frames = n_frames
    self.shape = frame_shape
    self.batch_size = batch_size
    self.shuffle = shuffle
#     file = open(train_list_path).readlines()
    self.videos = x_data
    self.labels = y_data
    self.flow_bound = flow_bound
    self.on_epoch_end()
    
  def on_epoch_end(self):
    if self.shuffle:
      temp = [i for i in range(len(self.labels))]
      random.shuffle(temp)
      self.videos = [self.videos[i] for i in temp]
      self.labels = [self.labels[i] for i in temp]
    

  def __getitem__(self, index):
    start_index = index * self.batch_size
    rgb = []
    flow = []
    y = []
    i = start_index - 1
    while len(y) < self.batch_size:
      try:
        i += 1
        print(self.videos[i % len(self.videos)], self.labels[i % len(self.videos)])
        video = capture_trim_resize(self.videos[i % len(self.videos)], self.shape, self.n_frames)
        op = tvl(video.copy(), self.flow_bound)
        rgb.append(video.astype(np.float32) / 255)
        flow.append(op)
        y.append(self.labels[i % len(self.videos)])
      except Exception as err:
        print(err)
        continue
    return {"input_rgb":np.array(rgb), "input_flow":np.array(flow)}, np.array(y)#, dtype=np.float32)

  def __len__(self):
    return int(np.floor(len(self.labels) / self.batch_size))

In [5]:
gen = DataGenerator(all_videos, all_labels, (224, 224), 256,1, False)
# gen2 = DataGenerator("videos", "testTrainMulti_7030_splits/testlist01.txt", (224, 224), 64, 1, False)

In [6]:
# import random
# gen.__getitem__(random.randint(0, 10000))

In [7]:
# path = ""
# all_videos = []
# all_labels = []
# for i in os.listdir(path):
#     for j in os.listdirdir(path _ '/' + i):
#         all_videos.append(path+'/' + i + '/' + j)
#         all_labels.append(i)

In [8]:
gen.videos = all_videos
gen.labels = all_labels
print(len(gen.videos), len(gen.labels))

97210 97210


In [9]:
size = (224, 224)
def ep(start, end):
    for i in range(start, end):
      a = gen.__getitem__(i)
      for i in range(len(a[1])):
        print(a[0]["input_rgb"].shape)
        print(a[0]["input_flow"].shape)
        print(a[1])
      c = str(a[1][0])
      f = str(int(len(os.listdir("dataset/" + c))))
      result = cv2.VideoWriter('dataset/' + c + '/rgb_' + f + '.avi', 
                             cv2.VideoWriter_fourcc(*'MJPG'),
                             10, size)
      for i in range(1):  
        for j in range(len(a[0]["input_rgb"][0])):
          frame = np.array(a[0]["input_rgb"][i][j] * 255, dtype=np.uint8)
          result.write(frame)
      result.release()
      result = cv2.VideoWriter('dataset/' + c + '/flow_' + f + '.avi', 
                              cv2.VideoWriter_fourcc(*'MJPG'),
                              10, size)
      for i in range(1):    
        for j in range(len(a[0]["input_rgb"][0])):
          frame = np.array(a[0]["input_flow"][i][j] * 255, dtype=np.uint8)
          result.write(frame)
      result.release()

In [ ]:
import multiprocessing
p = []
for i in range(1, 20000, 5000):
    p.append(multiprocessing.Process(target=ep, args=(i, min(i + 5000, len(all_videos)))))
for i in range(len(p)):
    p[i].start()
for i in range(len(p)):
    p[i].join()

/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_5rG8KbTTGvQ.avi catching or throwing frisbee
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_4j0q4XS8HOg.avi crawling baby
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_YApjDoBwszA.avi playing bass guitar
/home/uas-dtu/Desktop/Kinetics/trimming trees/vid_lRYpKDps_AY.avi trimming trees
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_JwRxJMSQ9-M.avi catching or throwing frisbee
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_1AHgbWyZO48.avi crawling baby
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_T1dYAUYLUa4.avi catching or throwing frisbee
CUDA runtime implicit initiali

CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/trimming trees/vid_P4L3nKijzV4.avi trimming trees
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_K_Kd_76EoT0.avi playing bass guitar
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_8-MvLqubskc.avi playing bass guitar
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_I6MwAnw80hA.avi crawling baby
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_fpQiF6k71TE.avi catching or throwing frisbee
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas

CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_Jn7TdLwQRS4.avi playing bass guitar
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_I9ZRSnrSaaw.avi crawling baby
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/trimming trees/vid_3Zo0cGD-PhU.avi trimming trees
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_a5dhsFbhfd8.avi catching or throwing frisbee
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/trimming trees/vid_FsohiksKkxQ.avi trimming trees
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desk

/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_Pv5-txE8iHs.avi crawling baby
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_0K1GHOu4viA.avi playing bass guitar
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_4DFLccHMXwQ.avi catching or throwing frisbee
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/trimming trees/vid_yOzRvtAFgsI.avi trimming trees
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid__xFVsnZ16_M.avi crawling baby
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_mxS8lTMTxBU.avi playing bass guitar
CUDA r

CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/trimming trees/vid_NEVYlUvj7M0.avi trimming trees
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_4wl_Yzvnops.avi playing bass guitar
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/trimming trees/vid_n6xSGqasdjQ.avi trimming trees
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_AQyzeVaED48.avi catching or throwing frisbee
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_a5tP_JWUfU0.avi playing bass guitar
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/u

...

/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_VAOSp9V4u1Y.avi crawling baby
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/trimming trees/vid_9sRNhHAQ8NE.avi trimming trees
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_eNmNCNRE930.avi playing bass guitar
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_6KxeHXVzAJk.avi crawling baby
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_gkcRul8yPhw.avi catching or throwing frisbee
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_K33pEsFxTg8.avi playing bass guitar
CUDA r

(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_X3fnRLfcNaM.avi catching or throwing frisbee
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_e_K8SDB_2nM.avi crawling baby
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_LUvok_MNL3M.avi playing bass guitar
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/cleaning windows/vid_c-jzRDkGty8.avi cleaning windows
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_OtuB-up-Rxw.avi catching or throwing frisbee
CUDA runtime implicit initialization on GPU:0 failed. Status: 

['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_fVBGOm-PnVE.avi catching or throwing frisbee
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_F18HqdylOYU.avi playing bass guitar
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/cleaning windows/vid_RjrMXPapPT4.avi cleaning windows
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_SWKz0N0TrGQ.avi playing bass guitar
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_Y4FFc3C_nws.avi crawling baby
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throw

(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_KqbItrWmXog.avi catching or throwing frisbee
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_Vp8ty3UCtyA.avi playing bass guitar
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_QdJliwXC2SM.avi crawling baby
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/cleaning windows/vid_o7F7A2C8eDc.avi cleaning windows
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_PxgTqgVMEXc.avi catching or throwing frisbee
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/ua

CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_Lihq_WmOI5o.avi playing bass guitar
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]
/home/uas-dtu/Desktop/Kinetics/crawling baby/vid_EED7ShdNStA.avi crawling baby
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/cleaning windows/vid_M8xAKPWF-hM.avi cleaning windows
CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory
/home/uas-dtu/Desktop/Kinetics/playing bass guitar/vid_M72xJDS8ZrM.avi playing bass guitar
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid__vH2vjBepGs.avi catching or throwing frisbee
OOM when allocating tensor with shape[256,224,224,3] and type float on /job:local

In [51]:
all_videos[0]

'/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_HBooKfe8G44.avi'

In [52]:
ep(0, 1)

/home/uas-dtu/Desktop/Kinetics/catching or throwing frisbee/vid_HBooKfe8G44.avi catching or throwing frisbee
(1, 256, 224, 224, 3)
(1, 256, 224, 224, 3)
['catching or throwing frisbee']
